In [1]:
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
import numpy as np
from collections import Counter
import sys

In [2]:
iris = datasets.load_iris()
x_train,x_test,y_train,y_test = train_test_split(iris.data,iris.target,random_state=1)

In [3]:
x_train[len(x_train)-1:]

array([[4.9, 3.6, 1.4, 0.1]])

In [4]:
# implementation of DecisionTree class
class DecisionTree:
    level = 0
    entropy = 0
    info_gain = 0
    gain_ratio = 0
    class_counts_dict = {}
    is_leaf_node = False
    children = []
    
    def __init__(self,level,entropy):
        self.level = level
        self.entropy = entropy
    
    def set_gain(self,info_gain):
        self.info_gain = info_gain
    
    def set_gain_ratio(self,gain_ratio):
        self.gain_ratio = gain_ratio
    
    def class_counts(self,class_counts_dict):
        for c in class_counts_dict.keys():
            self.class_counts_dict[c] = class_counts_dict[c]
    
    def set_is_leaf_node(self, is_leaf_node):
        self.is_leaf_node = is_leaf_node
    
    def set_children(self, children):
        self.children = children
        
    def print_node(self):
        print("Level ",self.level)
        for cls in self.class_counts_dict.keys():
            print("Count of",end=" ")
            print(cls,end=" = ")
            print(self.class_counts_dict[cls])
        print("Current Entropy  is = ",self.entropy)
        if self.is_leaf_node == True:
            print("Rached Leaf Node")
        else:
            print("gain {}".format(self.info_gain))
            print("gain ratio {}".format(self.gain_ratio))
            for c in self.children:
                c.print_node()

In [5]:
# Method to get Acuracy
def getAcuracy(y):
    classes = set(y)
    d = Counter(y)
    max_occur = 0
    for c in classes:
        if d[c]>max_occur:
            max_occur = d[c]
    return max_occur

In [6]:
# method to get Split Info
def splitInfo(x,feature):
    split_info = 0
    D = len(x)
    splits = set(x[:,feature])
    for s in splits:
        split_x = x[x[:,feature]==s]
        Di = len(split_x)
        split_info+=((-Di/D)*np.log2(Di/D))
    return split_info

In [7]:
# Method to get Entropy or Information gain or original
def entropy(y):
    classes = set(y)
    entropy = 0
    N = len(y)
    d = Counter(y)
    for c in classes:
        c_count = d[c]
        class_prob = c_count / N
        entropy+=(-class_prob*np.log2(class_prob))
    return entropy

In [8]:
# Method to get Information Gain for given feature
def gain(x,y,feature):
    # get feature split info
    splits = set(x[:,feature])
    D = len(y)
    entropy_original = entropy(y)
    feature_entropy = 0 
    for s in splits:
        match_y_array = y[x[:,feature]==s]
        s_entropy = entropy(match_y_array)
        feature_entropy+=((len(match_y_array)/D)*s_entropy)
    entropy_gain = entropy_original - feature_entropy
    return entropy_gain

In [9]:
# Decision Tree Algorithm Implementation
def decisionTree(x,y,features,level):
    print("Level ",level)
    # get count of classes:
    d = Counter(y)
    class_count_dict = {}
    for i in set(y):
        print("Count of",end=" ")
        print(i,end=" = ")
        print(d[i])
        class_count_dict[i] = d[i]
    
    #get current entropy
    entpy = entropy(y)
    dt = DecisionTree(level,entpy)
    print("Current Entropy  is = ",entropy(y))
    dt.class_counts(class_count_dict)

    # check if it's pure node
    if len(set(y))==1:
        print("Reached leaf Node")
        dt.set_is_leaf_node(True)
        return dt
    # check if feature length is 0
    if features is None or len(features)==0:
        print("Reached leaf Node")
        dt.set_is_leaf_node(True)
        return dt
    max_f_gain = -sys.maxsize
    final_feature = 0
    for f in features:
        # calculate gain
        f_gain = gain(x,y,f)
#         print("f_gain for feature {} = ".format(f),f_gain)
        if f_gain>max_f_gain:
            max_f_gain = f_gain
            final_feature = f
    print("Splitting on feature  {}  with gain {}".format(final_feature,max_f_gain))
    dt.set_gain(max_f_gain)
    # Gain Ratio
    split_info_v = splitInfo(x,final_feature)
    gain_ratio = max_f_gain / split_info_v
    print("split_info ",split_info_v)
    print("gain ratio ",gain_ratio)
    dt.set_gain_ratio(gain_ratio)
    # split the final feature selected
    splits = set(x[:,final_feature])
    # remove final_feature from existing feature list to send further
    final_features = features.copy()
    print(final_features)
    final_features.remove(final_feature)
    children = []
    # call decision Tree for each splitted feature
    for s in splits:
        split_x = x[x[:,final_feature]==s]
        split_y = y[x[:,final_feature]==s]
        # call decision tree
        children.append(decisionTree(split_x,split_y,final_features,level+1))
    dt.set_children(children)
    return dt

In [10]:
# or tree
# [1],
x = np.array([[1,1],
              [0,1],
              [1,0],
              [0,0]])
y = np.array([1,1,1,0])
or_features = [0,1]
or_level = 0
or_dt = decisionTree(x,y,or_features,or_level)

Level  0
Count of 0 = 1
Count of 1 = 3
Current Entropy  is =  0.8112781244591328
Splitting on feature  0  with gain 0.31127812445913283
split_info  1.0
gain ratio  0.31127812445913283
[0, 1]
Level  1
Count of 0 = 1
Count of 1 = 1
Current Entropy  is =  1.0
Splitting on feature  1  with gain 1.0
split_info  1.0
gain ratio  1.0
[1]
Level  2
Count of 0 = 1
Current Entropy  is =  0.0
Reached leaf Node
Level  2
Count of 1 = 1
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 1 = 2
Current Entropy  is =  0.0
Reached leaf Node


In [11]:
# Decision Tree for Iris Data
features = [0,1,2,3]
level = 0
dt = decisionTree(x_train,y_train,features,level)



Level  0
Count of 0 = 37
Count of 1 = 34
Count of 2 = 41
Current Entropy  is =  1.5807197138422102
Splitting on feature  2  with gain 1.4378625709850674
split_info  4.967701487018387
gain ratio  0.2894422248886118
[0, 1, 2, 3]
Level  1
Count of 0 = 11
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 0 = 3
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 1 = 2
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 1 = 2
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 2 = 3
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 2 = 2
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 2 = 3
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 2 = 5
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 2 = 1
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 0 = 10
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Count of 2 = 2
Current Entropy  is =  0.0
Reached leaf Node
Level  1
Cou

In [12]:
# Print Decision Tree node for Iris Data
dt.print_node()

Level  0
Count of 0 = 1
Count of 1 = 3
Count of 2 = 2
Current Entropy  is =  1.5807197138422102
gain 1.4378625709850674
gain ratio 0.2894422248886118
Level  1
Count of 0 = 1
Count of 1 = 3
Count of 2 = 2
Current Entropy  is =  0.0
Rached Leaf Node
Level  1
Count of 0 = 1
Count of 1 = 3
Count of 2 = 2
Current Entropy  is =  0.0
Rached Leaf Node
Level  1
Count of 0 = 1
Count of 1 = 3
Count of 2 = 2
Current Entropy  is =  0.0
Rached Leaf Node
Level  1
Count of 0 = 1
Count of 1 = 3
Count of 2 = 2
Current Entropy  is =  0.0
Rached Leaf Node
Level  1
Count of 0 = 1
Count of 1 = 3
Count of 2 = 2
Current Entropy  is =  0.0
Rached Leaf Node
Level  1
Count of 0 = 1
Count of 1 = 3
Count of 2 = 2
Current Entropy  is =  0.0
Rached Leaf Node
Level  1
Count of 0 = 1
Count of 1 = 3
Count of 2 = 2
Current Entropy  is =  0.0
Rached Leaf Node
Level  1
Count of 0 = 1
Count of 1 = 3
Count of 2 = 2
Current Entropy  is =  0.0
Rached Leaf Node
Level  1
Count of 0 = 1
Count of 1 = 3
Count of 2 = 2
Current Entr